<a href="https://colab.research.google.com/github/Montse1708/Dashboard/blob/main/TI_1_Practica4_JessicaMontserratMoralesEnrique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4

**Nombre:** Jessica Montserrat Morales Enrique  
**e-mail:** jessica.morales5556@alumnos.udg.mx

## MODULES

In [4]:
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

##CLASSES

In [11]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

##FUNCTIONS

###Brownian Motion (BM)

In [ ]:
###############################################################################################
# Brownian Motion Trajectory
###############################################################################################
def BM_2d(n_steps=1000, speed=6, s_pos=[0,0]):
    """
    Arguments:
        n_steps:
        speed:
        s_pos:
    Returns:
        BM_2d_df:
    """
    # Init velocity vector
    velocity = Vec2d(speed,0)

    BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps-1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    return BM_2d_df

In [ ]:
#Verificar que la función es correcta con el gráfico de la Brownian Motion

#Plot
BM_2d_df_2 = BM_2d(2000, 9, [5, 7])

#Init figure
fig_BM_2d = go.Figure()

#Plot trajectory
fig_BM_2d.add_trace(go.Scatter(
                      x = BM_2d_df_2.x_pos,
                      y = BM_2d_df_2.y_pos,
                      marker = dict(size=2),
                      line = dict(width=1),
                      mode='lines',
                      name='BM_2d_trajectory',
                      showlegend=True
))

#Actualizar el diseño para agregar el título
fig_BM_2d.update_layout(title='Trayectoria de Brownian Motion en 2D')

#Mostrar el gráfico
fig_BM_2d.show()

###Correlated Random Walk (CRW)

In [ ]:
######################################################################
# Correlated Random Walks (CRW)
######################################################################

def CRW_2d(CRW_exponents=0.2, n_steps=1000, speed=5, s_pos=[0,0]):
    # Init velocity vector
    velocity = Vec2d(speed, 0)
    # Guardar las trayectorias en Pandas DataFrame
    CRW_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    # Concatenar
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps-1 ):
        # Rotación de la trayectoria
        turn_angle = wrapcauchy.rvs(CRW_exponents)
        velocity = velocity.rotated(turn_angle)

        temp_df= pd.DataFrame([{'x_pos': CRW_2d_df.x_pos[i] + velocity.x, 'y_pos': CRW_2d_df.y_pos[i] + velocity.y}])
        # Concatenar
        CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)
    return CRW_2d_df

In [ ]:
# Verificar que la función es correcta con el gráfico de Correlated Random Walk (CRW)
CRW_df = CRW_2d(0.8, 500, 5, [5, 7])

# Inicializar el gráfico
fig_CRW = go.Figure()

# Agregar la trayectoria al gráfico
fig_CRW.add_trace(go.Scatter(
    x = CRW_df.x_pos,
    y = CRW_df.y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode='lines',
    name='CRW_trajectory',
    showlegend=True
))

# Actualizar el diseño para agregar el título
fig_CRW.update_layout(title='Trayectoria de Correlated Random Walk (CRW) en 2D')

# Mostrar el gráfico
fig_CRW.show()

###Lévy Flight (LF)

In [26]:
######################################################################
# Lévy Flight (LF)
######################################################################
def LF_2d(Levy_exponent = 0.5, n_steps=1000, speed = 5, s_pos=[0,0], alpha = 1.5, beta = 0, loc=3.0):
  #Init DFs
  Levy_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
  Levy_df = pd.concat([Levy_df, temp_df], ignore_index=True)

  # Init velocity vector
  velocity = Vec2d(speed,0)

  for i in range(n_steps-1):
    # Select turn angle
    turn_angle = wrapcauchy.rvs(c=Levy_exponent)
    #step size
    step_size = levy_stable.rvs(alpha=alpha, beta=beta, loc=loc)

    velocity = velocity.rotated(turn_angle)
    #Update position
    temp_df = pd.DataFrame([{'x_pos': Levy_df.x_pos[i]+(velocity.x*step_size), 'y_pos': Levy_df.y_pos[i]+(velocity.y*step_size)}])
    Levy_df = pd.concat ([Levy_df,temp_df], ignore_index=True)
  return Levy_df

In [44]:
#verificamos que funcione
LF_2d_df = LF_2d(0.8, 1000, 5, [5, 7], 0.7, 1.0, 3.0)

fig_LF_2d = go.Figure()

fig_LF_2d.add_trace(go.Scatter(x= LF_2d_df.x_pos,
                              y= LF_2d_df.y_pos,
                              #z=np.linspace(0,1,n_steps),
                              marker= dict(size=2),
                              line=dict(width=1),
                              mode='lines',
                              name='LF_3d',
                              showlegend=True))

fig_LF_2d.update_layout(title_text='Levy flight in 3D')

fig_LF_2d.show()